# ISEL - CSM
## Semestre 22/23
# <br>
###  <center> 3º Trabalho Prático </center>
# </br>

Trabalho realizado por:
* Guilherme Prazeres, numero <b> 49777 </b>
* Tiago Figueiredo, numero <b> 49154 </b>
* Tomás Charrua, numero <b> 49709 </b>

<hr style="border:1px color:black"> </hr>

<center><h3>Introdução</h3></center>
O objetivo deste trabalho é implementar e analisar o desempenho dum codificador/descodificador JPEG sequencial
para compressão de imagens a tons de cinzento. Deve ter em consideração que as funções realizadas devem conter uma
descrição e recomenda-se o uso de células “Markdown” para o efeito. Os resultados obtidos devem estar claramente
apresentados.
<hr style="border:1px color:black"> </hr>

In [57]:
#%pip install opencv-python
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Tomás Charrua\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [58]:
# Importar Bibliotecas
import numpy as np
from time import time
import os
import cv2 as cv
from matplotlib import pyplot as plt

In [59]:
imagem = cv.imread("LenaGray.tif")
imagem = cv.cvtColor(imagem, cv.COLOR_BGR2GRAY)

In [60]:
imagem

array([[162, 162, 162, ..., 170, 155, 128],
       [162, 162, 162, ..., 170, 155, 128],
       [162, 162, 162, ..., 170, 155, 128],
       ...,
       [ 43,  43,  50, ..., 104, 100,  98],
       [ 44,  44,  55, ..., 104, 105, 108],
       [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8)

In [61]:
imagem.shape

(512, 512)

<h1>Exercicio 1 </h1>
Construa uma função (codificador) que para cada bloco de 8×8 da imagem original efetue a DCT bidimensional
e construa outra função (descodificador) que faça a DCT inversa. Ver funções: cv2.dct() e cv2.idct())
Visualize a imagem depois da DCT e verifique que a DCT inversa resulta na imagem original.

In [62]:
def acertar_tamanho_imagem(imagem):

    nlinhas = imagem.shape[0]
    ncolunas = imagem.shape[1]


    nlinhas_extra = 0 #número de linhas a adicionar para tornar a dimenão da imagem divisível por 8
    ncolunas_extra = 0 #número de colunas a adicionar para tornar a dimenão da imagem divisível por 8

    if np.remainder(nlinhas, 8) > 0:
        nlinhas_extra = 8 - (nlinhas % 8)
    if np.remainder(ncolunas, 8) > 0:
        ncolunas_extra = 8 - (ncolunas % 8)

    #nova imagem com as dimensões atualizadas
    nova_imagem = np.zeros((nlinhas + nlinhas_extra, ncolunas + ncolunas_extra))

    #coloca-se o valor da imagem original nas dimensões respetivas
    nova_imagem[:nlinhas, :ncolunas] = imagem

    ultima_col = imagem[:, 1]
    for i in range(ncolunas_extra):
        nova_imagem[:, ncolunas + i] = ultima_col

    ultima_linha = nova_imagem[-1, :]
    for j in range(nlinhas_extra):
        nova_imagem[nlinhas + j, :] = ultima_linha

    return nova_imagem, nlinhas_extra, ncolunas_extra


In [63]:
def revert_imagem(imagem, nlinhas_extra, ncolunas_extra):
    
    nlinhas = imagem.shape[0] - nlinhas_extra
    ncolunas = imagem.shape[1] - ncolunas_extra

    imagem_original = imagem[:nlinhas, :ncolunas]

    return imagem_original

In [64]:
resized_image, nlinhas_extra, ncolunas_extra = acertar_tamanho_imagem(imagem)
print(resized_image)
print("Nº de linhas adicionadas:", nlinhas_extra)
print("Nº de colunas adicionadas:", ncolunas_extra)

[[162. 162. 162. ... 170. 155. 128.]
 [162. 162. 162. ... 170. 155. 128.]
 [162. 162. 162. ... 170. 155. 128.]
 ...
 [ 43.  43.  50. ... 104. 100.  98.]
 [ 44.  44.  55. ... 104. 105. 108.]
 [ 44.  44.  55. ... 104. 105. 108.]]
Nº de linhas adicionadas: 0
Nº de colunas adicionadas: 0


In [65]:
def divide_imagem_8x8(imagem):
    nlinhas, ncolunas = imagem.shape

    num_blocos_linhas = nlinhas // 8
    num_blocos_colunas = ncolunas // 8

    imagem_dividida = np.zeros((num_blocos_linhas, num_blocos_colunas, 8, 8))

    for i in range(num_blocos_linhas):
        for j in range(num_blocos_colunas):
            bloco = imagem[i*8:(i+1)*8, j*8:(j+1)*8]
            imagem_dividida[i, j] = bloco

    return imagem_dividida

In [66]:
imagem_dividida = divide_imagem_8x8(imagem)
print(imagem_dividida)
print(imagem_dividida.shape)

#for i in range(blocos.shape[0]):
#    for j in range(blocos.shape[1]):
#        print(blocos[i, j])

[[[[162. 162. 162. ... 157. 163. 161.]
   [162. 162. 162. ... 157. 163. 161.]
   [162. 162. 162. ... 157. 163. 161.]
   ...
   [164. 164. 158. ... 159. 159. 160.]
   [160. 160. 163. ... 162. 159. 156.]
   [159. 159. 155. ... 159. 156. 157.]]

  [[165. 161. 162. ... 163. 160. 155.]
   [165. 161. 162. ... 163. 160. 155.]
   [165. 161. 162. ... 163. 160. 155.]
   ...
   [160. 160. 155. ... 154. 156. 154.]
   [159. 162. 156. ... 155. 156. 152.]
   [159. 161. 161. ... 153. 153. 151.]]

  [[157. 156. 161. ... 156. 154. 157.]
   [157. 156. 161. ... 156. 154. 157.]
   [157. 156. 161. ... 156. 154. 157.]
   ...
   [156. 156. 153. ... 153. 153. 150.]
   [153. 160. 154. ... 150. 155. 152.]
   [155. 154. 156. ... 153. 152. 152.]]

  ...

  [[125. 121. 129. ... 127. 119. 123.]
   [125. 121. 129. ... 127. 119. 123.]
   [125. 121. 129. ... 127. 119. 123.]
   ...
   [126. 126. 129. ... 125. 121. 123.]
   [123. 125. 125. ... 126. 126. 124.]
   [121. 127. 129. ... 128. 125. 125.]]

  [[114. 121. 116. ..

In [67]:
def reconstruir_imagem(imagem_8x8, nLinhas, nColunas):
    imagem_reconstruida = np.zeros((nLinhas, nColunas))
    num_blocos_linhas, num_blocos_colunas, _, _ = imagem_8x8.shape

    for i in range(num_blocos_linhas):
        for j in range(num_blocos_colunas):
            bloco = imagem_8x8[i, j]
            imagem_reconstruida[i*8:(i+1)*8, j*8:(j+1)*8] = bloco
    return imagem_reconstruida

In [68]:
imagem_reconstruida = reconstruir_imagem(imagem_dividida, imagem.shape[0], imagem.shape[1])
imagem_reconstruida
imagem_reconstruida.shape

(512, 512)

In [69]:
def dct_bloco(bloco):
    b = np.array(bloco)
    b = b - 128
    return np.round(cv.dct(b.astype(np.float32)), 2)


In [70]:
def dct_blocos(blocos):
    num_blocos = len(blocos)
    b = np.zeros_like(blocos)
    for i in range(num_blocos):
        b[i] = dct_bloco(blocos[i])
    return b


In [71]:
def reverter_DCT(blocos):
    num_blocos = len(blocos)
    blocos_destransformados = np.array(blocos)
    for i in range(num_blocos):
        b_trans = np.round(cv.idct(blocos[i].astype(np.float32)), 2) + 128
        blocos_destransformados[i] = b_trans
    return blocos_destransformados

In [72]:
print(imagem_dividida.shape)
imagem_dct = dct_blocos(imagem_dividida)

T_imagem_idct = reverter_DCT(imagem_dct)
T_imagem = reconstruir_imagem(T_imagem_idct, imagem.shape[0], imagem.shape[1]).astype('uint8')
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 15))
ax1.imshow(cv.cvtColor(imagem, cv.COLOR_BGR2RGB))
ax1.set_title("Imagem Original")
ax2.imshow(cv.cvtColor(T_imagem, cv.COLOR_BGR2RGB))
ax2.set_title("Imagem Depois da DCT inversa")

(64, 64, 8, 8)


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\dxt.cpp:4417: error: (-215:Assertion failed) type == CV_32FC1 || type == CV_64FC1 in function 'cv::dct'


# Exercício 2
Construa uma função (codificador) que para cada bloco de 8 × 8 de coeficientes da transformação efectuada faça
a divisão pela matriz de quantificação (tabela K1 no anexo da norma) multiplicada por um factor de qualidade q
(ver função quality_factor do ficheiro Tabelas_jpeg.ipynb). Veja a imagem com o conjunto dos blocos após
a quantificação. Construa uma função (descodificador) que realize a operação inversa da quantificação.
Junte estas funções às já realizadas e verifique para diferentes fatores de qualidade qual a SNR e veja a imagem
descodificada.

# Exercício 3
Construa uma função (codificador) que faça a codificação diferencial dos coeficientes DC após a quantificação.
Construa a função inversa para o descodificador


# Exercício 4
Construa uma função (codificador) que crie um array com a indexação em zig-zag dos coeficientes AC após a
quantificação e crie um array com os pares (zero run length, nonzero value).
Construa a função inversa para o descodificador.



# Exercício 5
Junte estas funções às já realizadas e veja a imagem descodificada.

# Exercício 6
Construa uma função que dados os arrays das alíneas anteriores use as tabelas do código de Huffman (tabela K3
e K5) e grave num ficheiro a sequência de bits correspondente. (não é necessário usar o formato JFIF)


# Exercício 7
Construa uma função que leia o ficheiro gravado e retorne os arrays com os coeficientes AC e DC.


# Exercício 8
Junte estas funções às já realizadas e veja a imagem descodificada.
Para diferentes factores de qualidade meça a relação sinal-ruído e a taxa de compressão obtida. Represente um
gráfico onde se apresente a taxa de compressão em função do SNR.



# Exercício 9
No mesmo gráfico compare o seu compressor de imagem com outros existentes para várias qualidades.


# Exercício 10
O Jupyter Notebook o deve conter uma descrição breve das funções realizadas e uma tabela com todos os
resultados da SNR, taxa de compressão, tempo de compressão e descompressão.


# Conclusões
...